#### Read In Cleaned Text

In [1]:
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')
y_train = pd.read_csv('y_train.csv')

#### Create Word2Vec Vector:

In [2]:
w2v_model = gensim.models.Word2Vec(X_train, size=100, 
                                   window=5,
                                   min_count=2)

Replace the words in each text message with the learned word vector

In [5]:
words = set(w2v_model.wv.index2word)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
  for ls in X_train['clean_text']], dtype='object')

X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
  for ls in X_test['clean_text']], dtype='object')

Average the word vectors for each (and assign a vector of zeros if the model
did not learn any of the words in the text message during training.

In [6]:
X_train_vect_avg = []
for v in X_train_vect:
  if v.size:
    X_train_vect_avg.append(v.mean(axis=0))
  else:
    X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
  if v.size:
    X_test_vect_avg.append(v.mean(axis=0))
  else:
    X_test_vect_avg.append(np.zeros(100, dtype=float))

Unaveraged version:

In [7]:
X_train_vect[0]

array([[-3.4513872e-03,  1.6053747e-03, -2.0347871e-03,  1.9640834e-03,
         8.4220764e-04,  2.6279949e-03, -3.4153315e-03, -1.6174184e-03,
        -1.5756816e-03,  1.7378476e-04, -4.3680794e-03, -1.5627798e-03,
         2.5084894e-03, -4.9011372e-03,  4.3441062e-03, -4.9461646e-04,
         2.7512170e-03, -3.1035307e-03,  2.4119439e-03,  2.6358496e-03,
         4.4836025e-03, -1.8368851e-03,  4.7459966e-04, -2.8423148e-03,
         1.8292767e-05, -4.6047494e-03, -2.2806169e-03, -1.3865184e-03,
         3.1308979e-03,  3.6377902e-03,  1.7692666e-03, -4.2868014e-03,
        -1.6041686e-03, -1.4459669e-03, -1.6461924e-03,  2.4166047e-03,
        -1.5926058e-03,  3.0497154e-03, -1.4673431e-03, -3.0645065e-03,
         1.8609864e-04, -3.9744205e-03,  4.1641234e-04,  3.9766282e-03,
        -2.7675608e-03,  3.4115769e-03, -4.9207043e-03,  4.4140136e-03,
         5.0571811e-04,  3.8543097e-03,  2.9867175e-03, -1.4001796e-03,
         4.2502093e-03,  3.4749659e-03, -4.1471627e-05, -2.84804

Averaged version:

In [8]:
X_train_vect_avg[0]

array([-2.5681928e-03,  8.6273008e-04, -1.7183034e-03,  1.1582550e-03,
        1.5306051e-03,  2.0110614e-03, -2.6228752e-03, -2.2262151e-03,
       -1.3368739e-03,  4.1823968e-04, -4.1920156e-03, -9.3653577e-04,
        2.9171922e-03, -4.2417189e-03,  2.6616813e-03, -5.3839840e-04,
        3.0933004e-03, -2.8401348e-03,  9.4901631e-04,  1.8169113e-03,
        2.7398474e-03, -2.4689690e-03,  4.9163751e-05, -1.4809460e-03,
       -7.6641125e-04, -4.2493572e-03, -1.9749328e-03, -2.0319514e-03,
        1.8722115e-03,  3.1356819e-03,  6.3289609e-04, -2.8545861e-03,
       -6.4751471e-04, -1.4657456e-03, -4.7683361e-04,  1.2467771e-03,
       -4.4001694e-04,  2.0721264e-03, -2.1191291e-03, -2.2029744e-03,
       -3.6969996e-04, -3.4256559e-03, -4.5601442e-05,  2.5256095e-03,
       -2.1073776e-03,  3.4068320e-03, -2.9924307e-03,  3.1001973e-03,
        6.8797031e-04,  3.5791032e-03,  3.1943810e-03, -6.5343833e-04,
        3.1695168e-03,  3.7171915e-03, -6.6576339e-04, -3.1700642e-03,
      

#### Fit Random Forest Classifier:

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

Prediction on text data

In [10]:
y_pred = rf_model.predict(X_test_vect_avg)

Evaluate the predictions of the model.


In [11]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Precision: {} / Recall: {} /Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred == y_test['label']).sum()/len(y_pred),3)
))

Precision: 0.7 / Recall: 0.226 /Accuracy: 0.879
